In [ ]:
!pip install huggingface_hub > /dev/null

In [75]:
from langchain.chat_models import ChatOpenAI
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text
from kor import create_extraction_chain
from langchain.llms import OpenAI

In [77]:
##llm = OpenAI(temperature=0.9)
from langchain import HuggingFaceHub

repo_id = "google/flan-t5-small" # See https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads for some other options
repo_id = "tiiuae/falcon-7b"
llm = HuggingFaceHub(repo_id=repo_id, model_kwargs={"temperature":0, "max_length":64})

In [78]:
llm

HuggingFaceHub(cache=None, verbose=False, callbacks=None, callback_manager=None, client=InferenceAPI(api_url='https://api-inference.huggingface.co/pipeline/text-generation/tiiuae/falcon-7b', task='text-generation', options={'wait_for_model': True, 'use_gpu': False}), repo_id='tiiuae/falcon-7b', task=None, model_kwargs={'temperature': 0, 'max_length': 64}, huggingfacehub_api_token=None)

In [79]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

examples = [
  {
    "question": "I want to know the wstETH balance of 0xFe8e15ae884524eFfc2fe91dF6f5BA40D8533A92 on Gnosis",
    "answer": "action:balance,token:wstETH,address:0xFe8e15ae884524eFfc2fe91dF6f5BA40D8533A92,chain:Gnosis,amount:100"
  },
  {
    "question":"I want to swap 100 XDAI for DAI on Gnosis",
    "answer":"action:swap,token1:XDAI,token2:DAI,chain:Gnosis,amount:100"
  },
  {
    "question": "I want to send or transfer 100 WETH to 0xFe8e15ae884524eFfc2fe91dF6f5BA40D8533A92 on Gnosis",
    "answer":"action:transfer,token:WETH,chain:Gnosis,address:0xFe8e15ae884524eFfc2fe91dF6f5BA40D8533A92,amount:100"
  },
  
]

In [81]:
example_prompt = PromptTemplate(input_variables=["question", "answer"], template="Question: {question}\n{answer}")

print(example_prompt.format(**examples[0]))

Question: I want to know the wstETH balance of 0xFe8e15ae884524eFfc2fe91dF6f5BA40D8533A92 on Gnosis
action:balance,token:wstETH,address:0xFe8e15ae884524eFfc2fe91dF6f5BA40D8533A92,chain:Gnosis,amount:100


In [82]:
example_prompt = PromptTemplate(input_variables=["question", "answer"], template="Question: {question}\n{answer}")

prompt = FewShotPromptTemplate(
    examples=examples, 
    example_prompt=example_prompt, 
    suffix="Question: {input}", 
    input_variables=["input"]
)

In [83]:
print(prompt.format(input="I want to swap 100 DAI to USDC on UniSwap"))


Question: I want to know the wstETH balance of 0xFe8e15ae884524eFfc2fe91dF6f5BA40D8533A92 on Gnosis
action:balance,token:wstETH,address:0xFe8e15ae884524eFfc2fe91dF6f5BA40D8533A92,chain:Gnosis,amount:100

Question: I want to swap 100 XDAI for DAI on Gnosis
action:swap,token1:XDAI,token2:DAI,chain:Gnosis,amount:100

Question: I want to send or transfer 100 WETH to 0xFe8e15ae884524eFfc2fe91dF6f5BA40D8533A92 on Gnosis
action:transfer,token:WETH,chain:Gnosis,address:0xFe8e15ae884524eFfc2fe91dF6f5BA40D8533A92,amount:100

Question: I want to swap 100 DAI to USDC on UniSwap
